# Actividad face_recognition

In [1]:
import face_recognition
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from matplotlib import pyplot as plt
import pandas as pd
from datetime import date
import os

In [2]:
# función para obtener una lista con los nombres de los archivos de las imagenes

def list_files_in_folder(folder_path):
    file_names = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            file_names.append(file)
    return file_names

In [4]:
# Subir una foto de prueba y aprende a como reconocerla
known_face_encodings = []
known_face_names = []

for i in list_files_in_folder('FotosPruebaFace_Recog'):
    face = face_recognition.load_image_file(f"FotosPruebaFace_Recog/{i}")
    face_face_encoding = face_recognition.face_encodings(face)[0]
    
    known_face_encodings.append(face_face_encoding)
    known_face_names.append(i[:-5])


#######################################

video_capture = cv2.VideoCapture(0)

process_this_frame = True
while True:
    # Se agarra un solo frame del video
    ret, frame = video_capture.read()

    # Solo se procesan algunos frames para ahorrar tiempo
    if process_this_frame:
        # Hacer el frame 1/4 de su tamaño original
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convertir la imagen de BGR color (OpenCV) a RGB color (face_recognition)
        rgb_small_frame = small_frame
        
        # Encontrar todas las caras y codigos en el frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            
            # Verifica si la cara hace match con alguna del dataset
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance = 0.6)
            name = "Unknown"

            # En caso de que tenga dudas, pone la cara mas cerca a la de alguna del dataset
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame
    ret, frame = video_capture.read()
            
    # Cambiar de tamaño de la imagen
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)


    # Mostrar resultados
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Volver a poner las imagenes en tamaño orignal antes de reducirla a 1/4 de su tamaño
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Poner la caja alrededor de las caras
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Poner los labels de las caras en la caja
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 